# corsikaBatchGenerator
Generates corsika input files for runs to be done in parallel

Author: Nik Korzoun

## Corsika Userguide
For looking at keywords
https://web.iap.kit.edu/corsika/usersguide/usersguide.pdf

## Input template
adjustments will be made by the script to this default template when iterating

### PANOSETI

In [11]:
inptemp='''RUNNR 1
EVTNR 1
NSHOW 1
PRMPAR 1
ERANGE 1E5 1E5
ESLOPE -2.5
THETAP 0. 60.
PHIP 0. 360.
SEED 200 0 0
SEED 202 0 0
SEED 204 0 0
SEED 206 0 0
ATMOD 1
MAGNET 25.2 40.88
ARRANG 12.77
ELMFLG F T
RADNKG 200.E2
FIXCHI 0.
HADFLG 0 0 0 0 0 2
QGSJET T 0
QGSSIG T
HILOW 100.
ECUTS 0.30 0.05 0.02 0.02
MUADDI F
MUMULT T
LONGI T 20. F F
MAXPRT 50
PAROUT F F
ECTMAP 1.E6
DEBUG F 6 F 1000000
DIRECT ./
USER user
HOST host
ATMOSPHERE 61 T
TELFIL ./DATbatch0.telescope
OBSLEV 1239.E2
CSCAT 1 200.E2 200.E2
CERFIL 0
CERSIZ 5.
CWAVLG 200. 700.
TELESCOPE 53.59E2 73.52E2 1E2 0.25E2
TELESCOPE 53.59E2 -80.48E2 1E2 0.25E2
TELESCOPE -107.18E2 6.95E2 1E2 0.25E2
EXIT'''

## Helper functions

In [12]:
# definition of line number in inp from keyword
keyword = {
    "EVTNR": 1,
    "NSHOW": 2,
    "PRMPAR":3,
    "ERANGE": 4,
    "THETAP": 6,
    "PHIP": 7,
    "SEED1": 8,
    "SEED2": 9,
    "SEED3": 10,
    "SEED4": 11,
    "TELFIL": 34,
    "OBSLEV": 35,
    "CSCAT": 36,
    "CERSIZ": 38,
    "TELESCOPE": 40
}

In [13]:
def makeInput(*args):
    """
    Generates the input for a single corsika run
    
    Parameters:
         *args -- lists of line number and replacement string for the input template
             i.e. [ [line1, replacementString1], [line2,replacementString2], ... ]
             If Null, returns the input template
        
    Returns:
        inp -- string with contents of corsika input file
        
    """
    # set input to template
    inp = inptemp
    
    # check *args not null
    if (args):
        lines = inp.splitlines()
        for arg in args:
            lines[arg[0]] = arg[1]
        inp = "\n".join(lines)
    return inp

In [14]:
def writeInput(filename, contents):
    """
    Writes contents into a file.

    Parameters:
        filename -- name of the file to write to
        contents -- string to write into the file
        
    
    """
    file = open("{}".format(filename), "w")
    file.write(contents)
    file.close()

## Generate runs

In [15]:
def genRuns(index, controller):
    """
    Writes corsika input files.

    Parameters:
        index -- index of the first corsika input file generated
        controller -- list of corsika input keywords to adjust 
                    see example in the 'input controller' notebook cell
        
    
    """ 
    # check there is no invalid keyword (has a definition in the dictionary)
    for item in controller:
        if (item[0] not in keyword):
            e ='{} is not valid keyword. Update keyword dictionary or change controller.'.format(item[0])
            return e
    
        # check lists of each argument is the same size for each individual keyword
        size = len(item[1])
        for i in range(1,len(item)):
            if (size != len(item[i]) ):
                e='Mismatch in number of values for arguments in keyword {}. Fix controller.'.format(item[0])
                return e
    
    # check lists of each argument is the same size among all keywords (do not include nested keywords), 
    nVals = len(controller[-1][-1]) 
    if (nVals == 0):
        e = '0 arguments found in controller.'
        return e
    
    for item in controller:
        if (len(item[-1]) != nVals):       
            e = 'Mismatch in number of values among keywords. Fix controller.'
            return e
    
    # write input files
    for i in range(0, nVals):
        contents = []
        for item in controller:
            nArgs = len(item)
            replacementString = item[0]
            if('SEED' in replacementString):
                replacementString = 'SEED'
            for j in range(1,nArgs):
                replacementString += ' '
                replacementString += item[j][i]

            line = keyword[item[0]]
            contents.append([line,replacementString])
        
        # increment TELFIL name
        contents.append([keyword["TELFIL"],'TELFIL ./DATbatch{}.telescope'.format(index)])
        inp = makeInput(*contents)
        filename = 'batch{}.inp'.format(index)
        writeInput(filename, inp)
        index += 1
        
    return 'Created {} input files'.format(nVals)

### Input controllers
Tunes parameters to be adjusted and lists the values each argument will take

In [16]:
# Format:


# controller = [
#     [keyword1, [arg1_val1, arg1_val2, ...], [arg2_val1, arg2_val2, ...], ...],
#     [keyword2, [arg1_val1, arg1_val2, ...], ...],
#     [keyword3, [arg1_val1, arg1_val2, ...], [arg2_val1, arg2_val2, ...], [arg3_val1, arg3_val2, ...], ...]
# ]


# Example:

# controller = [
#     ['ERANGE',['1E3','1E4','1E5'],['1E3','1E4','1E5']],
#     ['CSCAT', ['1','1','1'], ['0.','0.','0.'] ['0.','0.','0.']]
# ]

# This will create 3 input files. 

# Notice that CSCAT is being kept constant but needs to have the same number of values for each argument
# as the other keywords. For this reason, it is easier to adjust CSCAT in the template rather than in
# the controller. This is particularly useful for the TELESCOPE line.

In [17]:
#controller = [
#    ['ERANGE',
#     ['1E3','3E3','5E3','1E4','3E4','5E4','1E5'],
#     ['1E3','3E3','5E3','1E4','3E4','5E4','1E5']
#    ],
#    ['SEED1',
#     ['200','208','216','224','232','240','248'],
#     ['0','0','0','0','0','0','0'],
#     ['0','0','0','0','0','0','0']
#    ],
#    ['SEED2',
#     ['202','210','218','226','234','242','250'],
#     ['0','0','0','0','0','0','0'],
#     ['0','0','0','0','0','0','0']
#    ],
#    ['SEED3',
#     ['204','212','220','228','236','244','252'],
#     ['0','0','0','0','0','0','0'],
#     ['0','0','0','0','0','0','0']
#    ],
#    ['SEED4',
#     ['206','214','222','230','238','246','254'],
#     ['0','0','0','0','0','0','0'],
#     ['0','0','0','0','0','0','0']
#    ]
#]

In [18]:
#controller = [
#    ['CSCAT',
#     ['10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10', '10'],
#     ['41000', '44000', '47000', '50000', '53000', '56000', '59000', '62000', '65000', '68000', '71000', '74000', '77000', '80000'],
#     ['41000', '44000', '47000', '50000', '53000', '56000', '59000', '62000', '65000', '68000', '71000', '74000', '77000', '80000']
#    ]
#]

In [19]:
import numpy as np

nFiles=10 #number of input files
nShow=1 #number of showers per run


seed1=['{}'.format(np.random.randint(0,9999)) for i in range(nFiles)]
seed2=['{}'.format(np.random.randint(0,9999)) for i in range(nFiles)]
seed3=['{}'.format(np.random.randint(0,9999)) for i in range(nFiles)]
seed4=['{}'.format(np.random.randint(0,9999)) for i in range(nFiles)]

controller = [
    ['EVTNR',[str(x) for x in range(1,(nFiles*nShow)+1,nShow)]],
    ['SEED1',seed1,[str(0) for x in range(nFiles)],[str(0) for x in range(nFiles)]],
    ['SEED2',seed2,[str(0) for x in range(nFiles)],[str(0) for x in range(nFiles)]],
    ['SEED3',seed3,[str(0) for x in range(nFiles)],[str(0) for x in range(nFiles)]],
    ['SEED4',seed4,[str(0) for x in range(nFiles)],[str(0) for x in range(nFiles)]]
]

# nFiles runs, nShow showers each run, change seed each file
# make sure NSHOW in the template file agrees with nShow above

In [20]:
genRuns(1, controller)

'Created 10 input files'